In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import file_io
import plot_utils
import sklearn
import sklearn.decomposition
import sklearn
import glob


from IPython.display import HTML

In [ ]:
def onoff_from_binary(data, return_duration=True):
    """Converts a binary variable data into onsets, offsets, and optionally durations

    This may yield unexpected behavior if the first value of `data` is true.

    Parameters
    ----------
    data : array-like, 1D
        binary array from which onsets and offsets should be extracted

    """
    data = data.astype(np.float).copy()
    ddata = np.hstack([[0], np.diff(data)])
    (onsets,) = np.nonzero(ddata > 0)
    # print(onsets)
    (offsets,) = np.nonzero(ddata < 0)
    # print(offsets)
    onset_first = onsets[0] < offsets[0]
    len(onsets) == len(offsets)

    on_at_end = False
    on_at_start = False
    if onset_first:
        if len(onsets) > len(offsets):
            offsets = np.hstack([offsets, [-1]])
            on_at_end = True
    else:
        if len(offsets) > len(onsets):
            onsets = np.hstack([-1, offsets])
            on_at_start = True
    onoff = np.vstack([onsets, offsets])
    if return_duration:
        duration = offsets - onsets
        if on_at_end:
            duration[-1] = len(data) - onsets[-1]
        if on_at_start:
            duration[0] = offsets[0] - 0
        onoff = np.vstack([onoff, duration])

    onoff = onoff.T.astype(np.int)
    return onoff

For this notebook, we will be using labeled blink data from gaze in wild. This is one session from Gaze in Wild, with a variety of labels.

In [ ]:
label_file = '/home/data/gaze_in_wild/16_1_1.mat'
eye_video_file = '/home/data/gaze_in_wild/16_1_1_ds.mp4'
pupillometry = dict(np.load('/home/data/gaze_in_wild/pupillometry.npz'))

These files aren't exactly like files we've dealt with before; two of them store arrays. file_io has a useful function to show what variables are in each.

In [ ]:
# file_io has a useful function to show what 
file_io.file_array_keys(label_file)

In [ ]:
list(pupillometry.keys())

The idea here will be to use pupil radius and pupil confidence (from our computed pupil estimate) to predict when blinks occur. 

In [ ]:
blink_labels = file_io.load_array(label_file, variable_name='blink').flatten()

In [ ]:
plt.plot(blink_labels)

Let's zoom a little so we can see what's going on - we'll only plot a chunk of the labels that are near 20000 frames:

In [ ]:
blink_region_st = 20000-500
blink_region_fin = 20000+500
plt.plot(blink_labels[blink_region_st:blink_region_fin])

This is a useful function to convert binary labels for every timepoint into indices for when the label turns on and turns off. 

In [ ]:
blink_onset_offset = onoff_from_binary(blink_labels)

In [ ]:
len(blink_onset_offset)

In [ ]:
# The values here are onset frame, offset frame, and duration (in frames)
blink_onset_offset[0]

These are blinks, right? Let's check!

In [ ]:
frames = file_io.load_mp4(eye_video_file, 
                          size=(30,40), 
                          frames=blink_onset_offset[2][:2], 
                          color='gray')

anim = plot_utils.make_image_animation(frames, 
                                       figsize=(3,3),
                                       cmap='gray')
HTML(anim.to_html5_video())

Construct a negative training set: what are NOT blinks? 

In [ ]:
negative_examples = []
for on, off, duration in blink_onset_offset:
    good = False
    while not good:
        start = np.random.randint(low=0, high=len(blink_labels)-duration)
        fin = start + duration
        good = np.sum(blink_labels[start:fin]) == 0
    negative_examples.append([start, fin, duration])
    

In [ ]:
frames = file_io.load_mp4(eye_video_file, 
                          size=(30,40), 
                          frames=negative_examples[77][:2], 
                          color='gray')

anim = plot_utils.make_image_animation(frames, 
                                       figsize=(3,3),
                                       cmap='gray')
HTML(anim.to_html5_video())

Not a blink. 

Quick check of how pupilradius matches up with blink epochs for the same time window we looked at above: 

In [ ]:
plt.plot(pupillometry['pupil_radius'][blink_region_st:blink_region_fin], '--', label='pupil radius')
# * 10 is just so you can see where the labels are more clearly. 
plt.plot(blink_labels[blink_region_st:blink_region_fin] * 10, label='blink labels')
plt.legend()

In [ ]:
plt.plot(pupillometry['pupil_confidence'][blink_region_st:blink_region_fin], '--', label='pupil confidence')
plt.plot(blink_labels[blink_region_st:blink_region_fin], label='blink labels')
plt.legend()

Make a classifier based on pupil confidence and eye radius

In [ ]:
# We will do the straightforward, perhaps naive thing, and try to 
# predict every time point that is labeled as a blink
positive_trials = [blink_labels[on:off] for on, off, _ in blink_onset_offset]
positive_trials = np.hstack(positive_trials)
negative_trials = [blink_labels[on:off] for on, off, _ in negative_examples]
negative_trials = np.hstack(negative_trials)

In [ ]:
plt.plot(np.hstack([positive_trials, negative_trials]))

In [ ]:
# We will do the straightforward, perhaps naive thing, and try to 
# predict every time point that is labeled as a blink
pupil_radius = pupillometry['pupil_radius']
radius_positive = [pupil_radius[on:off] for on, off, _ in blink_onset_offset]
radius_positive = np.hstack(radius_positive)
radius_negative = [pupil_radius[on:off] for on, off, _ in negative_examples]
radius_negative = np.hstack(radius_negative)

In [ ]:
# We will do the straightforward, perhaps naive thing, and try to 
# predict every time point that is labeled as a blink
pupil_confidence = pupillometry['pupil_confidence']
confidence_positive = [pupil_confidence[on:off] for on, off, _ in blink_onset_offset]
confidence_positive = np.hstack(confidence_positive)
confidence_negative = [pupil_confidence[on:off] for on, off, _ in negative_examples]
confidence_negative = np.hstack(confidence_negative)

In [ ]:
len(positive_trials), len(negative_trials)

In [ ]:
from sklearn import svm

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_positive = len(positive_trials)
n_negative = len(negative_trials) # should be the same! 

trial_labels = np.hstack([positive_trials, negative_trials])

trial_features = np.vstack([np.hstack([radius_positive, radius_negative]),
                           np.hstack([confidence_positive, confidence_negative])]).T

# create a k-fold object
skf = StratifiedKFold(n_splits=5)

overallAccuracy = np.zeros((5,))

foldCount = -1
# for each train-test split, print the length of train and test indices
for train_idx, test_idx in skf.split(trial_features, trial_labels):
    #print(train_idx.shape)
    #print(test_idx.shape) #pass # print(len(trainIdx), len(testIdx))
    foldCount += 1
    
    # initialize vector that will hold a 1 (accurate) or a 0 (inaccurate) value for each of the test items
    foldAccuracy = np.zeros((len(test_idx),))
    
    # define training data: fill in square brackets to indicate the trial indices for the given fold for training
    trainData = trial_features[train_idx]
    # define training answers
    trainAnswers = trial_labels[train_idx]
    
    # define testing data: fill in square brackets to indicate the trial indices for the given fold for testing
    testData = trial_features[test_idx]
    # define testing answers
    testAnswers = trial_labels[test_idx]
    
    print(testData.shape)
    # define the classifier
    classifier = svm.SVC(kernel = 'linear')
    
    # train the classifier
    classifier.fit(trainData, trainAnswers)
    
    # test the classifier
    predClass = classifier.predict(testData)
    
    # loop through each of the predicted classes and test whether they are correct
    for i in range(len(predClass)):
        if predClass[i]==testAnswers[i]: 
            foldAccuracy[i] = 1
            
    overallAccuracy[foldCount] = np.mean(foldAccuracy)

In [ ]:
overallAccuracy